In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import re

In [2]:
# --- 1. Database Connection Details ---

DB_USER = "root"
DB_PASS = "YourPassword"
DB_HOST = "localhost"
DB_PORT = "3306"
DB_NAME = "zomato_db"

In [3]:
# Create the database engine

engine = create_engine(f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}")

In [4]:
# --- 2. Load the Raw Data ---

file_path = "C:/Users/vaibh/Desktop/DA Project/ZomatoSQL/zomato.csv"

print("📥 Loading raw data from CSV...")
try:
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"ERROR: The file was not found at '{file_path}'. Please update the file_path variable.")
    exit()

print("🧹 Cleaning and preparing data...")

📥 Loading raw data from CSV...
🧹 Cleaning and preparing data...


In [5]:
# --- 3. Data Cleaning and Preprocessing ---

# Clean column names
df.columns = [col.lower().replace(' ', '_') for col in df.columns]
df.rename(columns={'approx_cost(for_two_people)': 'cost_for_two', 'listed_in(type)': 'restaurant_type'}, inplace=True)

# Drop unnecessary columns and initial duplicates
df.drop(columns=['url', 'dish_liked', 'phone', 'menu_item', 'listed_in(city)', 'reviews_list'], inplace=True, errors='ignore')
df.dropna(subset=['name'], inplace=True)
df.drop_duplicates(inplace=True)

In [6]:
# --- 4. Prepare and Clean the 'restaurants' DataFrame ---
# This section now isolates and fully cleans the data for the restaurants table.
restaurants_df = df[['name', 'address', 'location', 'rest_type', 'online_order', 'book_table', 'rate', 'votes', 'cost_for_two']].copy()
restaurants_df.rename(columns={
    'rest_type': 'restaurant_type',
    'online_order': 'has_online_order',
    'book_table': 'has_table_booking',
    'rate': 'rating'
}, inplace=True)

# Clean 'rating' column
restaurants_df['rating'] = restaurants_df['rating'].astype(str).str.replace('/5', '').str.strip()
restaurants_df['rating'] = pd.to_numeric(restaurants_df['rating'], errors='coerce')

# Clean 'cost_for_two' column
restaurants_df['cost_for_two'] = restaurants_df['cost_for_two'].astype(str).str.replace(',', '', regex=False)
restaurants_df['cost_for_two'] = pd.to_numeric(restaurants_df['cost_for_two'], errors='coerce')

# Clean boolean columns
restaurants_df['has_online_order'] = restaurants_df['has_online_order'].apply(lambda x: True if x == 'Yes' else False)
restaurants_df['has_table_booking'] = restaurants_df['has_table_booking'].apply(lambda x: True if x == 'Yes' else False)

# Add a temporary ID for linking later
restaurants_df['temp_id'] = range(1, len(restaurants_df) + 1)

In [7]:
# --- 5. Prepare the 'cuisines' and 'restaurant_cuisines' DataFrames ---
df.dropna(subset=['cuisines'], inplace=True)

all_cuisines = set()
for cuisine_list in df['cuisines'].str.split(','):
    all_cuisines.update([c.strip() for c in cuisine_list])

cuisines_df = pd.DataFrame(all_cuisines, columns=['cuisine_name'])
cuisines_df.sort_values('cuisine_name', inplace=True) # Sort for consistent IDs
cuisines_df['cuisine_id'] = range(1, len(cuisines_df) + 1)

cuisine_map = pd.Series(cuisines_df.cuisine_id.values, index=cuisines_df.cuisine_name).to_dict()

# Create a temporary mapping from original df index to restaurant temp_id
df.reset_index(inplace=True)
temp_id_map = df[['index']].copy()
temp_id_map['temp_id'] = range(1, len(temp_id_map) + 1)

restaurant_cuisines_records = []
for index, row in df.iterrows():
    temp_id = temp_id_map.iloc[index]['temp_id']
    cuisines = [c.strip() for c in row['cuisines'].split(',')]
    for cuisine in cuisines:
        if cuisine in cuisine_map:
            cuisine_id = cuisine_map[cuisine]
            restaurant_cuisines_records.append({
                'temp_id': temp_id,
                'cuisine_id': cuisine_id
            })

restaurant_cuisines_df = pd.DataFrame(restaurant_cuisines_records)

In [8]:
# --- 6. Clear Existing Data and Load into MySQL ---
print("🗑️ Clearing existing data from tables...")
with engine.connect() as connection:
    connection.execute(text("SET FOREIGN_KEY_CHECKS = 0;"))
    connection.execute(text("TRUNCATE TABLE restaurant_cuisines;"))
    connection.execute(text("TRUNCATE TABLE cuisines;"))
    connection.execute(text("TRUNCATE TABLE restaurants;"))
    connection.execute(text("SET FOREIGN_KEY_CHECKS = 1;"))
    connection.commit()
print("✅ Tables cleared.")

print("🚚 Loading data into MySQL...")

# Load restaurants
restaurants_to_load = restaurants_df.drop(columns=['temp_id'])
restaurants_to_load.to_sql('restaurants', engine, if_exists='append', index=False, chunksize=500)
print("✅ 'restaurants' table populated.")

# Load cuisines
cuisines_to_load = cuisines_df[['cuisine_name']] # Only load the name, ID is auto-incremented
cuisines_to_load.to_sql('cuisines', engine, if_exists='append', index=False)
print("✅ 'cuisines' table populated.")

# Get the real, auto-generated IDs back from the database
db_restaurants = pd.read_sql("SELECT restaurant_id, name, location FROM restaurants", engine)
db_cuisines = pd.read_sql("SELECT cuisine_id, cuisine_name FROM cuisines", engine)

# Merge to get the real IDs
restaurants_with_real_ids = restaurants_df.merge(db_restaurants, on=['name', 'location'], how='left')
cuisines_df = cuisines_df.merge(db_cuisines, on='cuisine_name', how='left')

# Prepare the final linking table with the real restaurant_id and cuisine_id
final_linking_df = restaurant_cuisines_df.merge(restaurants_with_real_ids[['temp_id', 'restaurant_id']], on='temp_id')
final_linking_df = final_linking_df.merge(cuisines_df[['cuisine_id_x', 'cuisine_id_y']].rename(columns={'cuisine_id_x':'cuisine_id', 'cuisine_id_y':'real_cuisine_id'}), on='cuisine_id')
final_linking_df = final_linking_df[['restaurant_id', 'real_cuisine_id']].rename(columns={'real_cuisine_id':'cuisine_id'})
final_linking_df.dropna(inplace=True)
final_linking_df.drop_duplicates(inplace=True)
final_linking_df['restaurant_id'] = final_linking_df['restaurant_id'].astype(int)
final_linking_df['cuisine_id'] = final_linking_df['cuisine_id'].astype(int)

# Load the linking table
final_linking_df.to_sql('restaurant_cuisines', engine, if_exists='append', index=False, chunksize=1000)
print("✅ 'restaurant_cuisines' table populated.")

print("\n🎉 ETL process complete! Your database is fully populated and ready for analysis.")


🗑️ Clearing existing data from tables...
✅ Tables cleared.
🚚 Loading data into MySQL...
✅ 'restaurants' table populated.
✅ 'cuisines' table populated.
✅ 'restaurant_cuisines' table populated.

🎉 ETL process complete! Your database is fully populated and ready for analysis.
